## First coverage tests

We will initially make simple coverage tests in simple examples to see how our proposed test is doing

In [1]:
# importing libraries
from sklearn.datasets import load_boston
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from nonconformist.cp import IcpRegressor
from nonconformist.nc import NcFactory
from sklearn.model_selection import train_test_split
from scipy import stats

boston = load_boston()
idx = np.random.permutation(boston.target.size)

# Divide the data into proper training set, calibration set and test set
idx_train, idx_cal, idx_test = idx[:300], idx[300:399], idx[399:]

model = RandomForestRegressor()	# Create the underlying model
nc = NcFactory.create_nc(model)	# Create a default nonconformity function
icp = IcpRegressor(nc)			# Create an inductive conformal regressor

# Fit the ICP using the proper training set
icp.fit(boston.data[idx_train, :], boston.target[idx_train])

# Calibrate the ICP using the calibration set
icp.calibrate(boston.data[idx_cal, :], boston.target[idx_cal])

/home/kuben/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dat

In [2]:
boston.data.shape

(506, 13)

Creating our hypothesis object:

In [3]:
import os
os.chdir("/home/kuben/estatistica_UFSCAR/Doutorado/lcv")

In [4]:
from lcv.valid_pred_sets import Valid_pred_sets
# hypothesis object
hyp_obj = Valid_pred_sets(icp, 0.05)

In [5]:
# fitting and testing
hyp_obj.fit(boston.data[idx_test, :], boston.target[idx_test])

Valid_pred_sets(alpha=0.05,
                conf=IcpRegressor(condition=<function BaseIcp.__init__.<locals>.<lambda> at 0x7f1c1b8d2820>,
                                  nc_function=RegressorNc(err_func=<nonconformist.nc.AbsErrorErrFunc object at 0x7f1c4c5685e0>,
                                                          model=RegressorAdapter(fit_params={},
                                                                                 model=RandomForestRegressor()))))

In [6]:
hyp_obj.monte_carlo_test()

0.748

Testing quantile regression

In [5]:
# creating quantile regression object
from lcv.valid_pred_sets import LinearQuantileRegression
lqr = LinearQuantileRegression(coverage = 0.05)

idx_train, idx_test = idx[:399], idx[399:]

# fitting
lqr.fit(boston.data[idx_train, :], boston.target[idx_train])

LinearQuantileRegression(coverage=0.05)

In [7]:
hyp_quantile_obj = Valid_pred_sets(lqr, 0.05)
hyp_quantile_obj.fit(boston.data[idx_test, :], boston.target[idx_test])

Valid_pred_sets(alpha=0.05, conf=LinearQuantileRegression(coverage=0.05))

In [8]:
hyp_quantile_obj.monte_carlo_test()

0.797